In [7]:
import pandas as pd
import sqlite3
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import time
import logging

# Create target directories if not already present
os.makedirs(r"C:\Users\GURU PRAVEEN REDDY J\Projects\logs", exist_ok=True)
os.makedirs(r"C:\Users\GURU PRAVEEN REDDY J\Projects\db", exist_ok=True)

# Configure logging
logging.basicConfig(
    filename=r"C:\Users\GURU PRAVEEN REDDY J\Projects\logs\ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s-%(message)s",
    filemode="a"
)

# Create SQLite database connection
engine = create_engine("sqlite:///C:/Users/GURU PRAVEEN REDDY J/Projects/db/inventory.db")

# Function to ingest large CSV file in chunks
def ingest_large_csv(file_path, table_name, engine, chunksize=100000):
    try:
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
            chunk.to_sql(table_name, con=engine, if_exists="append", index=False)
            logging.info(f"Chunk {i + 1} ingested into {table_name}")
    except Exception as e:
        logging.error(f"Error ingesting {file_path}: {str(e)}")

# Main function to load all raw data files
def load_raw_data():
    source_dir = r"C:\Users\GURU PRAVEEN REDDY J\Desktop\Datasets\Vendor Data Project Datasets\data\Cleaned Datasets"
    start_time = time.time()
    logging.info("Starting data ingestion")

    for file in os.listdir(source_dir):
        if file.endswith(".csv"):
            filepath = os.path.join(source_dir, file)
            table_name = os.path.splitext(file)[0].replace(" ", "_").lower()  # clean table name

            logging.info(f"Ingesting file: {file} into table: {table_name}")
            print(f"Ingesting: {file}")

            ingest_large_csv(filepath, table_name, engine)

    end_time = time.time()
    total_time = round((end_time - start_time) / 60, 2)
    logging.info(f"Ingestion completed in {total_time} minutes")

# Run when script is executed
if __name__ == "__main__":
    load_raw_data()

In [8]:
# ============================================
# Connect to SQLite Database
# --------------------------------------------
# Establishes a connection to the SQLite database
# located in the 'db' directory. If the file
# 'inventory.db' does not exist, it will be created.
# ============================================
conn=sqlite3.connect(r"db\inventory.db")

In [3]:
# ============================================
# Retrieve Table Names from SQLite Database
# --------------------------------------------
# Executes a SQL query on the SQLite system table
# 'sqlite_master' to fetch the names of all tables
# present in the connected database.
# The result is stored as a pandas DataFrame.
# ============================================
tables=pd.read_sql_query("select name from sqlite_master WHERE type='table' ",conn)
tables

,name
0,begin_inventory
1,end_inventory
2,purchase_details
3,purchase_price_info
4,sales_info
5,vendor_invoice


### For Vendor_sales_summry begin inventory and end_inventory not using to analyze sales main tables which are sales,purchase,vendor tables
### we can use these two inventory tables to check inventory wise performance also

## All datafiles info in-detailed way

### 1.Begin_Inventory.csv
| Column Name     | Description                                                                 |
|-----------------|-----------------------------------------------------------------------------|
| InventoryId     | A unique identifier for the inventory entry.                                |
| Store           | Identifier of the store where the inventory is located.                     |
| City            | The name of the city where the store is situated.                           |
| Brand           | The brand name of the product.                                              |
| Description     | A description of the inventory product.                                     |
| Size in ml      | The volume of the product in milliliters.                                   |
| onHand          | The number of units available at the beginning of the inventory period.     |
| Price           | The per-unit price of the inventory product.                                |
| startDate       | The date when the opening inventory was recorded.                           |


### 2. End_Inventory.csv
| Column Name   | Description                                |
| ------------- | ------------------------------------------ |
| `InventoryId` | Unique identifier for each inventory item. |
| `Store`       | Store ID where the inventory is tracked.   |
| `City`        | City in which the store is located.        |
| `Brand`       | Identifier for the product's brand.        |
| `Description` | Description or name of the product.        |
| `Size in ml`  | Product volume in milliliters.             |
| `onHand`      | Number of units available at the end date. |
| `Price`       | Unit price of the product.                 |
| `endDate`     | Date the end inventory snapshot was taken. |

### 3.Purchase_price_info.csv
| Column Name     | Description                                      |
|-----------------|--------------------------------------------------|
| Brand           | Identifier for the brand.                        |
| Description     | Product description.                             |
| Size            | Original size format (e.g., 750mL, 1.5L).        |
| Volume          | Converted size in milliliters.                   |
| Price           | Listed unit price in the purchase price sheet.  |

### 4.purchase_details.csv
| Column Name    | Description                                                                 |
|----------------|-----------------------------------------------------------------------------|
| InventoryId    | Unique identifier for the inventory item.                                   |
| Store          | Identifier of the store where the purchase occurred.                        |
| Brand          | Brand name of the purchased product.                                        |
| Description    | Textual description of the product.                                         |
| Size           | Size of the product (e.g., 750mL, 1.5L).                                     |
| VendorNumber   | Unique identifier for the vendor or supplier.                               |
| VendorName     | Name of the vendor or supplier.                                             |
| PONumber       | Purchase Order number associated with the transaction.                      |
| PODate         | Date when the Purchase Order was created.                                   |
| ReceivingDate  | Date when the product was received by the store.                            |
| InvoiceDate    | Date on which the invoice was generated by the vendor.                      |
| PayDate        | Date on which the payment was made to the vendor.                           |
| PurchasePrice  | Price paid per unit at the time of purchase.                                |
| Quantity       | Number of units purchased.                                                  |
| Dollars        | Total cost for the purchased quantity (i.e., PurchasePrice × Quantity).     |
| Volume         | Volume of the product (in mL or L), used for standardization.               |


### 5.Vendor_invoice.csv
| Column Name     | Description                                      |
|-----------------|--------------------------------------------------|
| Brand           | Product brand identifier.                        |
| Description     | Name or description of the product.             |
| Size            | Product size from invoice (e.g., 750mL).         |
| Volume          | Size converted to milliliters.                   |
| PurchasePrice   | Unit purchase price from the vendor.             |
| VendorNumber    | Unique vendor identification number.             |
| VendorName      | Name of the vendor.                              |
| InvoiceDate     | Date when the invoice was issued.                |
| PONumber        | Purchase Order number associated with invoice.   |
| PODate          | Date when the purchase order was made.           |
| PayDate         | Date when the payment was processed.             |
| Quantity        | Quantity of items purchased.                     |
| Dollars         | Total cost of the line item.                     |
| Freight         | Freight/shipping charges associated.             |

### 6.Sales_Info.csv
| Column Name    | Description                                                                 |
|----------------|-----------------------------------------------------------------------------|
| InventoryId    | Unique identifier for the inventory item.                                   |
| Store          | Identifier of the store where the sale took place.                          |
| Brand          | Brand name of the sold product.                                             |
| Description    | Textual description of the product.                                         |
| Size           | Size of the product (e.g., 750mL, 1.5L).                                     |
| SalesQuantity  | Number of units sold.                                                       |
| SalesDollars   | Total revenue generated from the sale (i.e., SalesPrice × SalesQuantity).   |
| SalesPrice     | Price at which the product was sold per unit.                               |
| SalesDate      | Date when the sale transaction occurred.                                    |
| Volume         | Volume of the product (in mL or L), used for standardization.               |
| ExciseTax      | Total excise tax applied to the sale.                                       |
| VendorNo       | Unique identifier of the vendor or supplier.                                |
| VendorName     | Name of the vendor or supplier.                                             |





In [9]:
# Loop through each table and print its name, row count, and first 5 rows
for table in tables["name"]:
    print("*" * 50, table, "*" * 50)
    query = f"SELECT COUNT(*) FROM {table}"
    result = pd.read_sql(query, conn)
    print(result)
    display(pd.read_sql(f"select * from {table} limit 5",conn))

************************************************** begin_inventory **************************************************
   COUNT(*)
0    206529


,InventoryId,Store,City,Brand,Description,Size in ml,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750.0,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750.0,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750.0,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750.0,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750.0,6,34.99,2024-01-01


************************************************** end_inventory **************************************************
   COUNT(*)
0    224489


,InventoryId,Store,City,Brand,Description,Size in ml,onHand,Price,endDate,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750.0,11,12.99,2024-12-31,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750.0,7,36.99,2024-12-31,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750.0,7,38.99,2024-12-31,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750.0,4,34.99,2024-12-31,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750.0,7,14.99,2024-12-31,2024-12-31


************************************************** purchase_details **************************************************
   COUNT(*)
0   2372318


...

,Brand,Description,Price,Size,Volume,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,10.94,7245,PROXIMO SPIRITS INC.


************************************************** sales_info **************************************************
   COUNT(*)
0  12825308


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,0.79,12546,JIM BEAM BRANDS COMPANY


************************************************** vendor_invoice **************************************************
   COUNT(*)
0      5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20


In [19]:
# ============================================
# Freight Summary Calculation per Vendor
# --------------------------------------------
# Aggregates total quantity, cost, and freight;
# then calculates unit purchase price per vendor.
# ============================================

freight_summary=pd.read_sql('''
        WITH CTE AS(SELECT 
        VendorNumber,
        VendorName,
        SUM(Quantity) TotalPurchasedQuantity,
        SUM(Dollars) Totalcost,
        SUM(Freight) TotalFreightCost
        FROM Vendor_invoice
        GROUP BY VendorNumber,VendorName)

        SELECT*,
        ROUND(CAST(Totalcost as FLOAT)/TotalPurchasedQuantity,2) AS UnitPurchasePrice
        FROM CTE
        ''',conn)                 

In [20]:
freight_summary

,VendorNumber,VendorName,TotalPurchasedQuantity,Totalcost,TotalFreightCost,UnitPurchasePrice
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",328,5630.88,27.08,17.17
1,54,AAPER ALCOHOL & CHEMICAL CO,1,105.07,0.48,105.07
2,60,ADAMBA IMPORTS INTL INC,4732,76770.25,367.52,16.22
3,105,ALTAMAR BRANDS LLC,332,11706.20,62.39,35.26
4,200,AMERICAN SPIRITS EXCHANGE,132,1205.16,6.19,9.13
...,...,...,...,...,...,...
123,98450,Serralles Usa LLC,10463,168993.61,856.02,16.15
124,99166,STARK BREWING COMPANY,1212,25961.04,130.09,21.42
125,172662,SWEETWATER FARM,1629,34708.03,178.34,21.31
126,173357,TAMWORTH DISTILLING,1990,41036.44,202.50,20.62


In [4]:
# ============================================
# Purchase Summary by Vendor and Brand
# --------------------------------------------
# Summarizes quantity & cost by brand and joins
# with volume/actual price from price info table.
# ============================================

purchase_summary=pd.read_sql('''
        WITH CTE_1 as
        (SELECT
        VendorName,
        VendorNumber,
        Brand,
        Description,
        PurchasePrice,
        SUM(Quantity) TotalPurchaseQuanity,
        SUM(Dollars) TotalPurchaseDollars
        FROM Purchase_details
        GROUP BY VendorName,VendorNumber,Brand,Description,PurchasePrice)

        SELECT
        c.VendorName,
        c.VendorNumber,
        c.Brand,
        c.Description,
        c.PurchasePrice,
        p.Volume,
        p.Price as ActualPurchasePrice,
        c.TotalPurchaseQuanity,
        c.TotalPurchaseDollars
        FROM CTE_1 as c
        JOIN purchase_price_info as p
        ON c.Brand=p.Brand
        ORDER BY c.TotalPurchaseDollars
        
        ''',conn)

In [49]:
purchase_summary

,VendorName,VendorNumber,Brand,PurchasePrice,Volume,ActualPurchasePrice,TotalPurchaseQuanity,TotalPurchaseDollars
0,PROXIMO SPIRITS INC.,7245,3065,0.71,50,0.99,1,0.71
1,DIAGEO NORTH AMERICA INC,3960,6127,1.47,200,1.99,1,1.47
2,HEAVEN HILL DISTILLERIES,3924,9123,0.74,50,0.99,2,1.48
3,SAZERAC CO INC,8004,5683,0.39,50,0.49,6,2.34
4,WINE GROUP INC,9815,8527,1.32,750,4.99,2,2.64
...,...,...,...,...,...,...,...,...
10687,DIAGEO NORTH AMERICA INC,3960,3545,21.89,1750,29.99,138109,3023206.01
10688,DIAGEO NORTH AMERICA INC,3960,4261,16.17,1750,22.99,201682,3261197.94
10689,PERNOD RICARD USA,17035,8068,18.24,1750,24.99,187407,3418303.68
10690,MARTIGNETTI COMPANIES,4425,3405,23.19,1750,28.99,164038,3804041.22


In [21]:
# ============================================
# Sales Summary by Vendor and Brand
# --------------------------------------------
# Aggregates sales metrics like quantity, revenue,
# excise tax, and average price per brand/vendor.
# ============================================

sales_summary=pd.read_sql('''
            SELECT
            VendorNO,
            Brand,
            AVG(SalesPrice) as AvgSalesPrice,
            SUM(SalesPrice) TotalSalesPrice,
            SUM(SalesQuantity) as TotalSalesQuantity,
            SUM(SalesDollars) TotalSalesDollars,
            SUM(ExciseTax) TotalExciseTax
            FROM sales_info
            GROUP BY VendorNO,Brand
            ORDER BY SalesDollars
            ''',conn)

In [22]:
sales_summary

,VendorNo,Brand,AvgSalesPrice,TotalSalesPrice,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax
0,8004,5287,0.49,0.98,2,0.98,0.10
1,480,4251,0.99,1969.11,5066,5015.34,262.26
2,480,4252,0.99,3051.18,9958,9858.42,516.74
3,480,4379,0.99,2604.69,5811,5752.89,299.23
4,480,4959,0.99,5420.25,17575,17399.25,913.70
...,...,...,...,...,...,...,...
11267,8004,2961,89.99,89.99,78,7019.22,61.42
11268,8004,2983,89.99,89.99,93,8369.07,73.24
11269,2000,20063,2199.99,6599.97,8,17599.92,0.90
11270,10754,17407,799.99,1599.98,13,10399.87,1.46


In [46]:
import time
start_time=time.time()
sales_vendor_summary=pd.read_sql('''
            WITH CTE_1 AS (
            SELECT
                    p.VendorNumber,
                    p.VendorName,
                    p.Brand,
                    p.Description,
                    p.PurchasePrice,
                    pp.Price as ActualPrice,
                    pp.Volume,
                    SUM(Quantity) as TotalPurchaseQuantity,
                    SUM(Dollars) as TotalPurchaseDllars
                FROM purchase_details as p
                JOIN purchase_price_info as pp
                ON p.Brand=pp.Brand
                WHERE p.PurchasePrice>0
                GROUP BY p.VendorNumber,p.VendorName,p.Brand,p.Description,p.PurchasePrice
            ),
            
            CTE_2 AS (
                SELECT
                    VendorNO,
                    Brand,
                    AVG(SalesPrice) AS AvgSalesPrice,
                    SUM(SalesPrice) AS TotalSalesPrice,
                    SUM(SalesQuantity) AS TotalSalesQuantity,
                    SUM(SalesDollars) AS TotalSalesDollars,
                    SUM(ExciseTax) AS TotalExciseTax
                FROM sales_info
                GROUP BY VendorNO, Brand
            ),
            
            CTE_3 AS (
                SELECT 
                    VendorNumber,
                    SUM(Freight) AS TotalFreightCost
                FROM Vendor_invoice
                GROUP BY VendorNumber
            )
            
            SELECT 
                c1.VendorNumber,
                c1.VendorName,
                c1.Brand,
                c1.Description,
                c1.PurchasePrice,
                c1.ActualPrice,
                c1.Volume,
                c1.TotalPurchaseQuantity,
                c1.TotalPurchaseDllars,
                c2.TotalSalesQuantity,
                c2.TotalSalesPrice,
                c2.TotalSalesDollars,
                c2.TotalExciseTax,
                c3.TotalFreightCost,
                c2.TotalSalesDollars-c1.TotalPurchaseDllars as Gross_Profit,
                ROUND((CAST(c2.TotalSalesDollars AS FLOAT)-c1.TotalPurchaseDllars)*100/c2.TotalSalesDollars,2) AS Profit,
                ROUND(CAST(c2.TotalSalesQuantity AS FLOAT)/c1.TotalPurchaseQuantity,2) AS StockTurnOver,
                ROUND(CAST(c2.TotalSalesDollars AS FLOAT)/c1.TotalPurchaseDllars,2) AS SalesToPurchaseRatio
    
            FROM CTE_1 AS c1
            LEFT JOIN CTE_2 AS c2
                ON c1.VendorNumber = c2.VendorNO AND c1.Brand = c2.Brand
            LEFT JOIN CTE_3 AS c3
                ON c1.VendorNumber = c3.VendorNumber
            ORDER BY c1.TotalPurchaseDllars DESC;
            ''',conn)
end_time=time.time()
print("total time took to run the query in seconds=",(end_time-start_time))

total time took to run the query in seconds= 55.93073749542236


In [47]:
sales_vendor_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
PurchasePrice              0
ActualPrice                0
Volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDllars        0
TotalSalesQuantity       178
TotalSalesPrice          178
TotalSalesDollars        178
TotalExciseTax           178
TotalFreightCost           0
Gross_Profit             178
Profit                   178
StockTurnOver            178
SalesToPurchaseRatio     178
dtype: int64

In [48]:
sales_vendor_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10691 entries, 0 to 10690
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorNumber           10691 non-null  int64  
 1   VendorName             10691 non-null  object 
 2   Brand                  10691 non-null  int64  
 3   Description            10691 non-null  object 
 4   PurchasePrice          10691 non-null  float64
 5   ActualPrice            10691 non-null  float64
 6   Volume                 10691 non-null  object 
 7   TotalPurchaseQuantity  10691 non-null  int64  
 8   TotalPurchaseDllars    10691 non-null  float64
 9   TotalSalesQuantity     10513 non-null  float64
 10  TotalSalesPrice        10513 non-null  float64
 11  TotalSalesDollars      10513 non-null  float64
 12  TotalExciseTax         10513 non-null  float64
 13  TotalFreightCost       10691 non-null  float64
 14  Gross_Profit           10513 non-null  float64
 15  Pr

In [49]:
sales_vendor_summary.fillna(0,inplace=True)

In [52]:
sales_vendor_summary.describe().T

,count,mean,std,min,25%,50%,75%,max
VendorNumber,10691.0,10650.752221,18754.393269,2.00,3951.000,7153.00,9552.000,201359.00
Brand,10691.0,18039.478907,12662.752886,58.00,5792.000,18762.00,25514.500,90631.00
PurchasePrice,10691.0,24.378994,109.272538,0.36,6.840,10.45,19.470,5681.81
ActualPrice,10691.0,35.633630,148.249314,0.49,10.990,15.99,28.990,7499.99
TotalPurchaseQuantity,10691.0,3141.179310,11095.564477,1.00,36.000,262.00,1976.500,337660.00
TotalPurchaseDllars,10691.0,30109.492434,123073.215326,0.71,453.995,3655.70,20743.070,3811251.60
TotalSalesQuantity,10691.0,3077.759891,10953.324922,0.00,33.000,261.00,1929.500,334939.00
TotalSalesPrice,10691.0,18795.491028,44954.529189,0.00,289.710,2857.80,16064.835,672819.31
TotalSalesDollars,10691.0,42242.962187,167662.625503,0.00,729.270,5298.21,28399.960,5101919.51
TotalExciseTax,10691.0,1774.383797,10976.081158,0.00,4.800,46.59,418.820,368242.80


In [ ]:
# ============================================
# Vendor Sales Summary Generator
# --------------------------------------------
# Combines purchase, sales, and freight data
# to calculate vendor-wise profitability metrics.
# ============================================

import pandas as pd
from sqlalchemy import create_engine
import os
import time
import logging

os.makedirs(r"C:\Users\GURU PRAVEEN REDDY J\Projects\logs", exist_ok=True)

logging.basicConfig(
    filename=r"C:\Users\GURU PRAVEEN REDDY J\Projects\logs\get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s-%(message)s",
    filemode="a"
)

def create_vendor_summary(conn):
    Vendor_sales_summary=pd.read_sql('''
            WITH CTE_1 AS (
            SELECT
                    p.VendorNumber,
                    p.VendorName,
                    p.Brand,
                    p.Description,
                    p.PurchasePrice,
                    pp.Price as ActualPrice,
                    pp.Volume,
                    SUM(Quantity) as TotalPurchaseQuantity,
                    SUM(Dollars) as TotalPurchaseDllars
                FROM purchase_details as p
                JOIN purchase_price_info as pp
                ON p.Brand=pp.Brand
                WHERE p.PurchasePrice>0
                GROUP BY p.VendorNumber,p.VendorName,p.Brand,p.Description,p.PurchasePrice
            ),
            
            CTE_2 AS (
                SELECT
                    VendorNO,
                    Brand,
                    AVG(SalesPrice) AS AvgSalesPrice,
                    SUM(SalesPrice) AS TotalSalesPrice,
                    SUM(SalesQuantity) AS TotalSalesQuantity,
                    SUM(SalesDollars) AS TotalSalesDollars,
                    SUM(ExciseTax) AS TotalExciseTax
                FROM sales_info
                GROUP BY VendorNO, Brand
            ),
            
            CTE_3 AS (
                SELECT 
                    VendorNumber,
                    SUM(Freight) AS TotalFreightCost
                FROM Vendor_invoice
                GROUP BY VendorNumber
            )
            
            SELECT 
                c1.VendorNumber,
                c1.VendorName,
                c1.Brand,
                c1.Description,
                c1.PurchasePrice,
                c1.ActualPrice,
                c1.Volume,
                c1.TotalPurchaseQuantity,
                c1.TotalPurchaseDllars,
                c2.TotalSalesQuantity,
                c2.TotalSalesPrice,
                c2.TotalSalesDollars,
                c2.TotalExciseTax,
                c3.TotalFreightCost,
                c2.TotalSalesDollars-c1.TotalPurchaseDllars as Gross_Profit,
                ROUND((CAST(c2.TotalSalesDollars AS FLOAT)-c1.TotalPurchaseDllars)*100/c2.TotalSalesDollars,2) AS Profit,
                ROUND(CAST(c2.TotalSalesQuantity AS FLOAT)/c1.TotalPurchaseQuantity,2) AS StockTurnOver,
                ROUND(CAST(c2.TotalSalesDollars AS FLOAT)/c1.TotalPurchaseDllars,2) AS SalesToPurchaseRatio
    
            FROM CTE_1 AS c1
            LEFT JOIN CTE_2 AS c2
                ON c1.VendorNumber = c2.VendorNO AND c1.Brand = c2.Brand
            LEFT JOIN CTE_3 AS c3
                ON c1.VendorNumber = c3.VendorNumber
            ORDER BY c1.TotalPurchaseDllars DESC;
            ''',conn)
    return Vendor_sales_summary
    
def to_clean_data(df):
    df.fillna(0,inplace=True)
    return df

def ingest_db(df,table_name,engine):
    df.to_sql(table_name,con=engine,if_exists="replace",index=False)

if __name__ == "__main__":
    conn=sqlite3.connect(r"db\inventory.db")
    logging.info("Creating Vendor Summary table")
    summary_df=create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info("Cleaning data")
    clean_df=to_clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info("Ingesting To DB")
    ingest_db(clean_df,'Vendor_sales_summary',conn)
    logging.info("completed")
    